In [2]:
#Define the molecule
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

molecule = Molecule(
    geometry=[["He", [0.0, 0.0, 0.0]], ["He", [0.0, 0.512, 0.735]], ["He", [0.3, 0.418, 0.972]]], charge=4, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())


#Define the solver

from qiskit_nature.algorithms import NumPyEigensolverFactory

numpy_solver = NumPyEigensolverFactory(use_default_filter_criterion=True)


#Define qEOM as solver


from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory

# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))
solver = VQEUCCFactory(quantum_instance=quantum_instance)
gsc = GroundStateEigensolver(qubit_converter, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")



#Calculation and results

from qiskit_nature.algorithms import ExcitedStatesEigensolver

numpy_excited_states_calculation = ExcitedStatesEigensolver(qubit_converter, numpy_solver)
numpy_results = numpy_excited_states_calculation.solve(es_problem)

qeom_results = qeom_excited_states_calculation.solve(es_problem)

print(numpy_results)
print("\n\n")
print(qeom_results)


#Use of the custom filter function to check consistently our results and only filter out states with incorrect number of particle (in this case the number of particle is 2).

import numpy as np


def filter_criterion(eigenstate, eigenvalue, aux_values):
    return np.isclose(aux_values[0][0], 2.0)


new_numpy_solver = NumPyEigensolverFactory(filter_criterion=filter_criterion)
new_numpy_excited_states_calculation = ExcitedStatesEigensolver(qubit_converter, new_numpy_solver)
new_numpy_results = new_numpy_excited_states_calculation.solve(es_problem)

print(new_numpy_results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -9.847372612499
  - computed part:      -9.847372612499
~ Nuclear repulsion energy (Hartree): 9.664087446152
> Total ground state energy (Hartree): -0.183285166347
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -8.797495404204
> Total excited state energy (Hartree): 0.866592041947
  2: 
* Electronic excited state energy (Hartree): -7.603983682682
> Total excited state energy (Hartree): 2.060103763469
  3: 
* Electronic excited state energy (Hartree): -7.04753029061
> Total excited state energy (Hartree): 2.616557155542
  4: 
* Electronic excited state energy (Hartree): -6.511198794494
> Total excited state energy (Hartree): 3.152888651658
  5: 
* Electronic excited state energy (Hartree): -5.710040968783
> Total excited state energy (Hartree): 3.954046477368
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 0.0